# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [27]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [31]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
# 類別特徵選擇"Embarked" 登船的地方；數值特徵選擇"Fare" 票價
df['Embarked'] = df['Embarked'].fillna('None')
mean_df = df.groupby(['Embarked'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x:x.mode()[0]).reset_index()
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index()
max_df = df.groupby(['Embarked'])['Fare'].max().reset_index()
min_df = df.groupby(['Embarked'])['Fare'].min().reset_index()

temp = pd.merge(mean_df, mode_df, how='left', on='Embarked')
temp = pd.merge(temp, median_df, how='left', on='Embarked')
temp = pd.merge(temp, max_df, how='left', on='Embarked')
temp = pd.merge(temp, min_df, how='left', on='Embarked')

temp.columns = ['Embarked', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min']
temp

,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min
0,C,59.954144,7.2292,29.70,512.3292,4.0125
1,None,80.000000,80.0000,80.00,80.0000,80.0000
2,Q,13.276030,7.7500,7.75,90.0000,6.7500
3,S,27.079812,8.0500,13.00,263.0000,0.0000


In [33]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
# 類別特徵選擇"Sex" 性別；數值特徵選擇"Age" 年紀
df['Sex'] = df['Sex'].fillna('None')

mean_df2 = df.groupby(['Sex'])['Age'].mean().reset_index()
mode_df2 = df.groupby(['Sex'])['Age'].apply(lambda x:x.mode()[0]).reset_index()
median_df2 = df.groupby(['Sex'])['Age'].median().reset_index()
max_df2 = df.groupby(['Sex'])['Age'].max().reset_index()
min_df2 = df.groupby(['Sex'])['Age'].min().reset_index()

temp2 = pd.merge(mean_df2, mode_df2, how='left', on='Sex')
temp2 = pd.merge(temp2, median_df2, how='left', on='Sex')
temp2 = pd.merge(temp2, max_df2, how='left', on='Sex')
temp2 = pd.merge(temp2, min_df2, how='left', on='Sex')

temp2.columns = ['Sex', 'Age_mean', 'Age_mode', 'Age_median', 'Age_max', 'Age_min']
temp2

,Sex,Age_mean,Age_mode,Age_median,Age_max,Age_min
0,female,27.915709,24.0,27.0,63.0,0.75
1,male,30.726645,19.0,29.0,80.0,0.42


In [34]:
df = df.merge(temp, how='left', on='Embarked')
df.drop('Embarked', axis=1, inplace=True)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,27.079812,8.0500,13.0,263.0000,0.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,59.954144,7.2292,29.7,512.3292,4.0125
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,27.079812,8.0500,13.0,263.0000,0.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,27.079812,8.0500,13.0,263.0000,0.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,27.079812,8.0500,13.0,263.0000,0.0000


In [35]:
df = df.merge(temp2, how='left', on='Sex')
df.drop('Sex', axis=1, inplace=True)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Age_mean,Age_mode,Age_median,Age_max,Age_min
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,27.079812,8.0500,13.0,263.0000,0.0000,30.726645,19.0,29.0,80.0,0.42
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,59.954144,7.2292,29.7,512.3292,4.0125,27.915709,24.0,27.0,63.0,0.75
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,27.079812,8.0500,13.0,263.0000,0.0000,27.915709,24.0,27.0,63.0,0.75
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,27.079812,8.0500,13.0,263.0000,0.0000,27.915709,24.0,27.0,63.0,0.75
4,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,27.079812,8.0500,13.0,263.0000,0.0000,30.726645,19.0,29.0,80.0,0.42


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 18 columns):
Pclass         891 non-null int64
Name           891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Fare_mean      891 non-null float64
Fare_mode      891 non-null float64
Fare_median    891 non-null float64
Fare_max       891 non-null float64
Fare_min       891 non-null float64
Age_mean       891 non-null float64
Age_mode       891 non-null float64
Age_median     891 non-null float64
Age_max        891 non-null float64
Age_min        891 non-null float64
dtypes: float64(12), int64(3), object(3)
memory usage: 132.3+ KB


In [37]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

15 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Age_mean', 'Age_mode', 'Age_median', 'Age_max', 'Age_min']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Age_mean,Age_mode,Age_median,Age_max,Age_min
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000,0.0000,30.726645,19.0,29.0,80.0,0.42
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292,4.0125,27.915709,24.0,27.0,63.0,0.75
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000,0.0000,27.915709,24.0,27.0,63.0,0.75
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000,0.0000,27.915709,24.0,27.0,63.0,0.75
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000,0.0000,30.726645,19.0,29.0,80.0,0.42


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 15 columns):
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Fare_mean      891 non-null float64
Fare_mode      891 non-null float64
Fare_median    891 non-null float64
Fare_max       891 non-null float64
Fare_min       891 non-null float64
Age_mean       891 non-null float64
Age_mode       891 non-null float64
Age_median     891 non-null float64
Age_max        891 non-null float64
Age_min        891 non-null float64
dtypes: float64(12), int64(3)
memory usage: 111.4 KB


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

1.原始特徵 + 邏輯斯迴歸

scores =  0.7038635542329971

2.新特徵："Embarked" 登船的地方；數值特徵選擇"Fare" 票價 + 邏輯斯迴歸

scores =  0.7016289882888318

3.新特徵："Sex" 性別；數值特徵選擇"Age" 年紀 + 邏輯斯迴歸

scores =  0.7822774202448624

新增二組新的特徵，分別為     
(1)"Embarked" 登船的地方；數值特徵選擇"Fare" 票價，scores =  0.7016289882888318。

(2)"Sex" 性別；數值特徵選擇"Age" 年紀，scores =  0.7822774202448624。

就結果來說，(1)登船地方+票價，與原始特徵比較起來差異不大。
      
          (2)性別+票價，與原始特徵比較起來，則有明顯的增加。

In [40]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

df_minus = df.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 
                    'Fare_min','Age_mean','Age_mode','Age_median','Age_max','Age_min'], axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
scores = cross_val_score(estimator, train_X, train_Y, cv=5)
print('cross_val_score with original features + logistic regression.')
print('scores = ', scores)
print('mean validation scores = ', scores.mean())

cross_val_score with original features + logistic regression.
scores =  [0.62569832 0.68715084 0.70786517 0.74719101 0.75141243]
mean validation scores =  0.7038635542329971


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 15 columns):
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Fare_mean      891 non-null float64
Fare_mode      891 non-null float64
Fare_median    891 non-null float64
Fare_max       891 non-null float64
Fare_min       891 non-null float64
Age_mean       891 non-null float64
Age_mode       891 non-null float64
Age_median     891 non-null float64
Age_max        891 non-null float64
Age_min        891 non-null float64
dtypes: float64(12), int64(3)
memory usage: 111.4 KB


In [42]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

df_minus1 = df.drop(['Age_mean','Age_mode','Age_median','Age_max','Age_min'], axis=1)

train_X = MMEncoder.fit_transform(df_minus1)
estimator = LogisticRegression()
scores = cross_val_score(estimator, train_X, train_Y, cv=5)
print('cross_val_score with new features + logistic regression.')
print('scores = ', scores)
print('mean validation scores = ', scores.mean())

cross_val_score with new features + logistic regression.
scores =  [0.62569832 0.68156425 0.70224719 0.74157303 0.75706215]
mean validation scores =  0.7016289882888318


In [45]:
df_minus1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Fare_mean      891 non-null float64
Fare_mode      891 non-null float64
Fare_median    891 non-null float64
Fare_max       891 non-null float64
Fare_min       891 non-null float64
dtypes: float64(7), int64(3)
memory usage: 76.6 KB


In [47]:
df_minus2 = df.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max'], axis=1)

train_X = MMEncoder.fit_transform(df_minus2)
estimator = LogisticRegression()
scores = cross_val_score(estimator, train_X, train_Y, cv=5)
print('cross_val_score with new features + logistic regression.')
print('scores = ', scores)
print('mean validation scores = ', scores.mean())

cross_val_score with new features + logistic regression.
scores =  [0.79888268 0.78212291 0.7752809  0.74719101 0.8079096 ]
mean validation scores =  0.7822774202448624


In [48]:
df_minus2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 11 columns):
Pclass        891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Fare_min      891 non-null float64
Age_mean      891 non-null float64
Age_mode      891 non-null float64
Age_median    891 non-null float64
Age_max       891 non-null float64
Age_min       891 non-null float64
dtypes: float64(8), int64(3)
memory usage: 83.5 KB
